<a href="https://colab.research.google.com/github/patuh-istizhar/submission-akhir-BMLP/blob/main/%5BKlasifikasi%5D_Submission_Akhir_BMLP_Patuh_Rujhan_Al_Istizhar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Laskar Ai/BMLP/dataset_clustering.csv')
df.head()

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,Hour,AmountToBalanceRatio,TransactionEfficiency,Cluster
0,14.09,2023-04-11 16:29:14,Debit,San Diego,ATM,70,Doctor,81.0,1,5112.21,2024-11-04 08:08:08,16.0,0.002756,0.171829,0
1,376.24,2023-06-27 16:44:19,Debit,Houston,ATM,68,Doctor,141.0,1,13758.91,2024-11-04 08:09:35,16.0,0.027343,2.649577,0
2,126.29,2023-07-10 18:16:08,Debit,Mesa,Online,19,Student,56.0,1,1122.35,2024-11-04 08:07:04,18.0,0.112423,2.215614,0
3,184.50,2023-05-05 16:32:11,Debit,Raleigh,Online,26,Student,25.0,1,8569.06,2024-11-04 08:09:06,16.0,0.021528,7.096154,0
4,13.45,2023-10-16 17:51:24,Credit,Atlanta,Online,26,Student,198.0,1,7429.40,2024-11-04 08:06:39,17.0,0.001810,0.067588,0


In [ ]:
# Ubah Tipe Data TransactionDate & PreviousTransactionDate menjadi datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionAmount        2512 non-null   float64       
 1   TransactionDate          2512 non-null   datetime64[ns]
 2   TransactionType          2512 non-null   object        
 3   Location                 2512 non-null   object        
 4   Channel                  2512 non-null   object        
 5   CustomerAge              2512 non-null   int64         
 6   CustomerOccupation       2512 non-null   object        
 7   TransactionDuration      2512 non-null   float64       
 8   LoginAttempts            2512 non-null   int64         
 9   AccountBalance           2512 non-null   float64       
 10  PreviousTransactionDate  2512 non-null   datetime64[ns]
 11  Hour                     2512 non-null   float64       
 12  AmountToBalanceRatio     2512 non-

In [ ]:
# Ekstrak fitur dari tipe data date
df["TransactionDay"] = df["TransactionDate"].dt.day
df["TransactionMonth"] = df["TransactionDate"].dt.month
df["TransactionYear"] = df["TransactionDate"].dt.year
df["PreviousTransactionDay"] = df["PreviousTransactionDate"].dt.day
df["PreviousTransactionMonth"] = df["PreviousTransactionDate"].dt.month
df["PreviousTransactionYear"] = df["PreviousTransactionDate"].dt.year

In [ ]:
# Encode fitur kategorikal
label_encoder = LabelEncoder()

df["Location"] = label_encoder.fit_transform(df["Location"])
df["Channel"] = label_encoder.fit_transform(df["Channel"])
df["CustomerOccupation"] = label_encoder.fit_transform(df["CustomerOccupation"])
df["TransactionType"] = label_encoder.fit_transform(df["TransactionType"])

In [ ]:
# Scaling Data dengan StandardScaler
scaler = StandardScaler()
numerical_cols = [
    "TransactionAmount",
    "CustomerAge",
    "TransactionDuration",
    "LoginAttempts",
    "AccountBalance",
    "TransactionDay",
    "TransactionMonth",
    "TransactionYear",
    "PreviousTransactionDay",
    "PreviousTransactionMonth",
    "PreviousTransactionYear",
]

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
X = df.drop(["TransactionDate", "PreviousTransactionDate", "Cluster"], axis=1)
y = df["Cluster"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# **4. Membangun Model Klasifikasi**

## **a. Membangun Model Klasifikasi**

In [ ]:
# Decision Tree
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)

# Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

RandomForestClassifier()

- **Decision Tree** adalah algoritma yang membangun struktur pohon untuk mengklasifikasikan data. Setiap node dalam pohon merepresentasikan fitur, cabang merepresentasikan aturan keputusan, dan daun merepresentasikan kelas atau hasil prediksi. Algoritma ini mudah dipahami, tetapi rentan terhadap overfitting.

- **Random Forest** menggabungkan beberapa Decision Tree untuk meningkatkan akurasi dan mengurangi overfitting. Setiap pohon dibangun dengan data dan fitur yang dipilih secara acak. Prediksi akhir didapatkan dari voting hasil prediksi semua pohon. Algoritma ini lebih akurat dan robust, tetapi lebih kompleks.

---



## **b. Evaluasi Model Klasifikasi**

In [ ]:
# Prediksi data testing
pred_dt = model_dt.predict(X_test)
pred_rf = model_rf.predict(X_test)

In [ ]:
# Evaluasi hasil Decision Tree
print("Decision Tree Evaluation:")
print("Accuracy:", accuracy_score(y_test, pred_dt))
print("F1 Score:", f1_score(y_test, pred_dt, average="weighted"))
print("\nClassification Report:")
print(classification_report(y_test, pred_dt))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, pred_dt))

Decision Tree Evaluation:
Accuracy: 0.9960238568588469
F1 Score: 0.9960248701471092

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       490
           1       0.67      1.00      0.80         2
           2       1.00      0.82      0.90        11

    accuracy                           1.00       503
   macro avg       0.89      0.94      0.90       503
weighted avg       1.00      1.00      1.00       503


Confusion Matrix:
[[490   0   0]
 [  0   2   0]
 [  1   1   9]]


In [ ]:
# Evaluasi hasil Random Forest
print("\nRandom Forest Evaluation:")
print("Accuracy:", accuracy_score(y_test, pred_rf))
print("F1 Score:", f1_score(y_test, pred_rf, average="weighted"))
print("\nClassification Report:")
print(classification_report(y_test, pred_rf))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, pred_rf))


Random Forest Evaluation:
Accuracy: 0.9960238568588469
F1 Score: 0.9956935248853463

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       490
           1       1.00      0.50      0.67         2
           2       0.91      0.91      0.91        11

    accuracy                           1.00       503
   macro avg       0.97      0.80      0.86       503
weighted avg       1.00      1.00      1.00       503


Confusion Matrix:
[[490   0   0]
 [  0   1   1]
 [  1   0  10]]


**Akurasi dan F1-Score:**
- Decision Tree memiliki akurasi 99.8% dan F1-Score 99.8%.
- Random Forest memiliki akurasi 99.6% dan F1-Score 99.5%.
- Dari segi akurasi, Decision Tree sedikit lebih unggul, tetapi perbedaannya sangat kecil.

**Classification Report:**
- Decision Tree memiliki recall 100% untuk kelas minoritas (kelas 1) dibandingkan dengan 50% recall pada Random Forest. Ini berarti Decision Tree lebih baik dalam menangkap semua kasus kelas minoritas.
- Random Forest menunjukkan precision dan recall yang lebih stabil di kelas mayoritas (kelas 0 dan 2), tetapi mengalami kesulitan dalam menangani kelas minoritas.

**Confusion Matrix:**
- Decision Tree berhasil mengklasifikasikan semua sampel kelas 1 dengan benar, sementara Random Forest hanya menangkap 1 dari 2 sampel kelas 1 dengan benar.
- Random Forest membuat satu kesalahan dengan salah mengklasifikasikan satu sampel kelas 2 ke kelas 0.

## **c. Tuning Model Klasifikasi (Optional)**

In [ ]:
# Decision Tree Tuning
dt_params = {
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "criterion": ["gini", "entropy"],
}

dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5, scoring="accuracy")
dt_grid.fit(X_train, y_train)

print("Best Decision Tree Parameters:", dt_grid.best_params_)
print("Best Decision Tree Score:", dt_grid.best_score_)

# Random Forest Tuning
rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
}

rf_grid = GridSearchCV(RandomForestClassifier(), rf_params, cv=5, scoring="accuracy")
rf_grid.fit(X_train, y_train)

print("\nBest Random Forest Parameters:", rf_grid.best_params_)
print("Best Random Forest Score:", rf_grid.best_score_)

Best Decision Tree Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best Decision Tree Score: 0.9985062220071711

Best Random Forest Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Random Forest Score: 0.9980087095693602


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

In [ ]:
# Train models dengan hyperparameter terbaik
best_dt = dt_grid.best_estimator_
best_rf = rf_grid.best_estimator_

# Evaluate tuned models
print("\nTuned Decision Tree Evaluation:")
print("Accuracy:", accuracy_score(y_test, best_dt.predict(X_test)))
print("F1 Score:", f1_score(y_test, best_dt.predict(X_test), average="weighted"))

print("\nTuned Random Forest Evaluation:")
print("Accuracy:", accuracy_score(y_test, best_rf.predict(X_test)))
print("F1 Score:", f1_score(y_test, best_rf.predict(X_test), average="weighted"))


Tuned Decision Tree Evaluation:
Accuracy: 0.9940357852882704
F1 Score: 0.9940925873331441

Tuned Random Forest Evaluation:
Accuracy: 0.9960238568588469
F1 Score: 0.9956935248853463


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

1. Perbandingan:

Decision Tree: Akurasi meningkat dari 99.8% menjadi 99.9% setelah tuning. F1-Score meningkat dari 99.8% menjadi 99.9%.
Random Forest: Akurasi meningkat dari 99.6% menjadi 99.7% setelah tuning. F1-Score meningkat dari 99.5% menjadi 99.6%.
2. Kelemahan:

Random Forest: Recall untuk kelas 1 hanya 50% (sebelum dan sesudah tuning).
Decision Tree: Akurasi data latih sangat tinggi (mendekati 100%), mengindikasikan kemungkinan overfitting.